# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,vorgashor,67.5833,63.9500,5.31,97,12,4.03,RU,1713977284
1,1,korla,41.7597,86.1469,72.21,19,92,6.82,CN,1713977284
2,2,ushuaia,-54.8000,-68.3000,47.86,61,20,9.22,AR,1713977224
3,3,fellsmere,27.7678,-80.6014,79.99,51,0,1.01,US,1713977284
4,4,bethel,41.3712,-73.4140,67.62,45,0,9.22,US,1713977207


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
humidity_map = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', 
                                           hover_cols=['City', 'Humidity'], 
                                           size='Humidity', 
                                           title='Humidity Map',
                                           colorbar=True, 
                                           cmap='viridis', 
                                           width=800, height=500)

# Display the map
humidity_map


:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define the ideal weather conditions
ideal_conditions_df = city_data_df.loc[(city_data_df["Max Temp"] > 20) & 
                                       (city_data_df["Max Temp"] < 28) & 
                                       (city_data_df["Humidity"] < 60)]

# Display the DataFrame with ideal weather conditions
ideal_conditions_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
140,140,kapuskasing,49.4169,-82.4331,21.29,38,0,5.75,CA,1713977315
433,433,labrador city,52.9463,-66.9114,25.18,54,100,14.97,CA,1713977403


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a copy of the ideal_conditions_df DataFrame to create the hotel_df DataFrame
hotel_df = ideal_conditions_df.copy()

# Add a new column 'Hotel Name' to the hotel_df DataFrame
hotel_df['Hotel Name'] = ''

# Display the hotel_df DataFrame
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
140,140,kapuskasing,49.4169,-82.4331,21.29,38,0,5.75,CA,1713977315,
433,433,labrador city,52.9463,-66.9114,25.18,54,100,14.97,CA,1713977403,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Define the radius in meters
params = {
    "categories": "accommodation",  # Specify category as accommodation (hotels)
    "limit": 1,  # Limit the number of results to 1
    "radius": radius,  # Set the search radius
    "apiKey": geoapify_key  # Use your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{latitude},{longitude}"
    params["bias"] = f"point:{latitude},{longitude}"    
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapuskasing - nearest hotel: No hotel found
labrador city - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
140,140,kapuskasing,49.4169,-82.4331,21.29,38,0,5.75,CA,1713977315,No hotel found
433,433,labrador city,52.9463,-66.9114,25.18,54,100,14.97,CA,1713977403,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotel_map = hotel_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', 
                                    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'], 
                                    size='Humidity', 
                                    title='Hotel Map',
                                    colorbar=True, 
                                    cmap='viridis', 
                                    width=800, height=500)

# Display the map
hotel_map

:Scatter   [Lng]   (Lat,Humidity,City,Hotel Name,Country)